In [39]:
import pandas as pd
from pymongo import MongoClient

# Replace with your MongoDB connection string
connection_string = "mongodb://localhost:27017"

# Establish a connection to the MongoDB server
client = MongoClient(connection_string)

# Access the database and collection
db = client['secondCA']
collection = db['dataset']

# the whole dataset
cursor = collection.find()

# Convert the cursor to a list of dictionaries
data = list(cursor)

# Convert the list of dictionaries to a Pandas DataFrame
df = pd.DataFrame(data)

# Close the connection
client.close()

In [40]:
# Display the first 10 rows of the DataFrame
df.head(5)


,_id,index,ids,date,flag,user,text,timestamp,sentiment_score
0,664a0fb1e1eba02a48ec456e,20,1467813985,2009-04-07,NO_QUERY,quanvu,@alydesigns i was out most of the day so didn'...,2009-04-07 05:20:37,0.0000
1,664a0fb1e1eba02a48ec456f,22,1467814119,2009-04-07,NO_QUERY,cooliodoc,@angry_barista I baked you a cake but I ated it,2009-04-07 05:20:40,0.0000
2,664a0fb1e1eba02a48ec4570,37,1467817374,2009-04-07,NO_QUERY,ajaxpro,@MissXu sorry! bed time came here (GMT+1) ht...,2009-04-07 05:21:30,-0.1511
3,664a0fb1e1eba02a48ec4571,39,1467818007,2009-04-07,NO_QUERY,Anthony_Nguyen,Bed. Class 8-12. Work 12-3. Gym 3-5 or 6. Then...,2009-04-07 05:21:39,-0.1531
4,664a0fb1e1eba02a48ec4572,45,1467819650,2009-04-07,NO_QUERY,antzpantz,@Viennah Yay! I'm happy for you with your job!...,2009-04-07 05:22:05,0.6290


In [41]:
df.columns


Index(['_id', 'index', 'ids', 'date', 'flag', 'user', 'text', 'timestamp',
       'sentiment_score'],
      dtype='object')

In [45]:
df.dtypes

_id                        object
index                       int64
ids                         int64
date               datetime64[ns]
flag                       object
user                       object
text                       object
timestamp          datetime64[ns]
sentiment_score           float64
dtype: object

In [30]:
# Print all unique values in the "date" column
unique_dates = df['date'].unique()
unique_dates


<DatetimeArray>
['2009-04-07 00:00:00', '2009-04-18 00:00:00', '2009-04-19 00:00:00',
 '2009-04-20 00:00:00', '2009-04-21 00:00:00', '2009-05-02 00:00:00',
 '2009-05-03 00:00:00', '2009-05-04 00:00:00', '2009-05-10 00:00:00',
 '2009-05-11 00:00:00', '2009-05-12 00:00:00', '2009-05-14 00:00:00',
 '2009-05-17 00:00:00', '2009-05-18 00:00:00', '2009-05-22 00:00:00',
 '2009-05-24 00:00:00', '2009-05-25 00:00:00', '2009-05-27 00:00:00',
 '2009-05-29 00:00:00', '2009-05-30 00:00:00', '2009-05-31 00:00:00',
 '2009-06-01 00:00:00', '2009-06-02 00:00:00', '2009-06-03 00:00:00',
 '2009-06-04 00:00:00', '2009-06-05 00:00:00', '2009-06-06 00:00:00',
 '2009-06-07 00:00:00', '2009-06-08 00:00:00', '2009-06-14 00:00:00',
 '2009-06-15 00:00:00', '2009-06-16 00:00:00', '2009-06-17 00:00:00',
 '2009-06-18 00:00:00', '2009-06-19 00:00:00', '2009-06-20 00:00:00',
 '2009-06-21 00:00:00', '2009-06-22 00:00:00', '2009-06-23 00:00:00',
 '2009-06-24 00:00:00', '2009-06-25 00:00:00']
Length: 41, dtype: datetime

In [43]:
print(df.head(5))

                        _id  index         ids       date      flag  \
0  664a0fb1e1eba02a48ec456e     20  1467813985 2009-04-07  NO_QUERY   
1  664a0fb1e1eba02a48ec456f     22  1467814119 2009-04-07  NO_QUERY   
2  664a0fb1e1eba02a48ec4570     37  1467817374 2009-04-07  NO_QUERY   
3  664a0fb1e1eba02a48ec4571     39  1467818007 2009-04-07  NO_QUERY   
4  664a0fb1e1eba02a48ec4572     45  1467819650 2009-04-07  NO_QUERY   

             user                                               text  \
0          quanvu  @alydesigns i was out most of the day so didn'...   
1       cooliodoc    @angry_barista I baked you a cake but I ated it   
2         ajaxpro  @MissXu sorry! bed time came here (GMT+1)   ht...   
3  Anthony_Nguyen  Bed. Class 8-12. Work 12-3. Gym 3-5 or 6. Then...   
4       antzpantz  @Viennah Yay! I'm happy for you with your job!...   

            timestamp  sentiment_score  
0 2009-04-07 05:20:37           0.0000  
1 2009-04-07 05:20:40           0.0000  
2 2009-04-07 05:2

In [46]:
# Aggregate sentiment scores by date
daily_sentiment = df.groupby('date')['sentiment_score'].mean().reset_index()

In [47]:
# Set date as index
daily_sentiment.set_index('date', inplace=True)

In [48]:
# Create a complete date range
all_dates = pd.date_range(start=daily_sentiment.index.min(), end=daily_sentiment.index.max(), freq='D')


In [49]:
# Reindex DataFrame to include all dates
daily_sentiment = daily_sentiment.reindex(all_dates)

In [50]:
# Interpolate missing values
daily_sentiment['sentiment_score'] = daily_sentiment['sentiment_score'].interpolate(method='time')

In [60]:
# Ensure the DataFrame is now ready for ARIMA modeling
print(daily_sentiment.head(10))

            sentiment_score
2009-04-07         0.158750
2009-04-08         0.161215
2009-04-09         0.163680
2009-04-10         0.166145
2009-04-11         0.168609
2009-04-12         0.171074
2009-04-13         0.173539
2009-04-14         0.176004
2009-04-15         0.178468
2009-04-16         0.180933


In [53]:
!pip install pmdarima

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.0/628.0 kB 4.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 6.1 MB/s eta 0:00:0000:0100:01


In [54]:
import numpy as np
import pmdarima as pm

# Fit ARIMA model
model = pm.auto_arima(daily_sentiment['sentiment_score'], seasonal=False, stepwise=True)

/Users/fernandoleniz/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/fernandoleniz/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/fernandoleniz/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/fernandoleniz/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseD

/Users/fernandoleniz/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/fernandoleniz/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/fernandoleniz/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/fernandoleniz/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseD

In [55]:
# Forecast for 1, 3, and 7 days ahead
forecast_1_day = model.predict(n_periods=1)
print("1 day forecast:", forecast_1_day)

1 day forecast: 2009-06-26   -0.064643
Freq: D, dtype: float64


/Users/fernandoleniz/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/fernandoleniz/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/fernandoleniz/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/fernandoleniz/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a fu

In [56]:
forecast_3_days = model.predict(n_periods=3)
print("3 days forecast:", forecast_3_days)

3 days forecast: 2009-06-26   -0.064643
2009-06-27   -0.071226
2009-06-28   -0.066642
Freq: D, dtype: float64


/Users/fernandoleniz/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/fernandoleniz/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/fernandoleniz/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/fernandoleniz/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a fu

In [57]:
forecast_7_days = model.predict(n_periods=7)
print("7 days forecast:", forecast_7_days)

7 days forecast: 2009-06-26   -0.064643
2009-06-27   -0.071226
2009-06-28   -0.066642
2009-06-29   -0.066968
2009-06-30   -0.075687
2009-07-01   -0.080620
2009-07-02   -0.079244
Freq: D, dtype: float64


/Users/fernandoleniz/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/fernandoleniz/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/fernandoleniz/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/fernandoleniz/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a fu

In [63]:
# Define seasonal order (in this case: daily data with weekly seasonality)
seasonal_order = (1, 1, 1, 7)  # (P, D, Q, s)

# Fit SARIMA model
model_sarima = pm.auto_arima(daily_sentiment['sentiment_score'], 
                      seasonal=True, 
                      m=7,  # Weekly seasonality (7 days)
                      stepwise=True,
                      suppress_warnings=True)

/Users/fernandoleniz/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/fernandoleniz/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/fernandoleniz/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/fernandoleniz/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseD

/Users/fernandoleniz/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/fernandoleniz/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/fernandoleniz/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/fernandoleniz/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseD

In [64]:
# Forecast for 1 day in SARIMA
forecast_1_day_sarima = model_sarima.predict(n_periods=1)
print("1 day forecast using SARIMA:", forecast_1_day_sarima)

1 day forecast using SARIMA: 2009-06-26   -0.051698
Freq: D, dtype: float64


/Users/fernandoleniz/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/fernandoleniz/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/fernandoleniz/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/fernandoleniz/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a fu

In [65]:
forecast_3_days_sarima = model_sarima.predict(n_periods=3)
print("3 days forecast using SARIMA:", forecast_3_days_sarima)

3 days forecast using SARIMA: 2009-06-26   -0.051698
2009-06-27   -0.051698
2009-06-28   -0.051698
Freq: D, dtype: float64


/Users/fernandoleniz/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/fernandoleniz/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/fernandoleniz/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/fernandoleniz/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a fu

In [66]:
forecast_7_days_sarima = model_sarima.predict(n_periods=7)
print("7 days forecast using SARIMA:", forecast_7_days_sarima)

7 days forecast using SARIMA: 2009-06-26   -0.051698
2009-06-27   -0.051698
2009-06-28   -0.051698
2009-06-29   -0.051698
2009-06-30   -0.051698
2009-07-01   -0.051698
2009-07-02   -0.051698
Freq: D, dtype: float64


/Users/fernandoleniz/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/fernandoleniz/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/fernandoleniz/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/fernandoleniz/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a fu